[View in Colaboratory](https://colab.research.google.com/github/lyashevska/Complete-Python-Bootcamp/blob/master/SciPy_Cupy.ipynb)

In [1]:
# Install CuPy and NumPy
!apt -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 \
  /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install cupy-cuda80
import numpy as np

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcusparse8.0 is already the newest version (8.0.61-1).
libnvrtc8.0 is already the newest version (8.0.61-1).
libnvtoolsext1 is already the newest version (8.0.61-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [0]:
import time, numpy, cupy

In [0]:
def test(xp, size):
  return [xp.arange(size).reshape(1000, -1).T * 2 for x in range(10)]

In [4]:
for i in range(4,9):
  for xp in [numpy, cupy]:
    test(xp, 10**i)   # Avoid first call overhead
    # Synchronize CPU and GPU for benchmark
    cupy.cuda.runtime.deviceSynchronize()
    t1 = time.time()
    test(xp, 10 ** i)
    cupy.cuda.runtime.deviceSynchronize()
    t2 = time.time()
    print(xp.__name__, (t2 - t1) * 100)

numpy 0.09343624114990234
cupy 0.18956661224365234
numpy 0.4306316375732422
cupy 0.22037029266357422
numpy 5.22761344909668
cupy 0.4457712173461914
numpy 57.15289115905762
cupy 4.214787483215332
numpy 571.681547164917
cupy 75.15921592712402


In [9]:
import contextlib
import time

import numpy as np
from scipy.cluster.vq import kmeans
import six

import cupy

# Args
gpu_id = 0
n_clusters = 2
n_samples = 5000000
n_iter = 10


@contextlib.contextmanager
def timer(message):
    cupy.cuda.runtime.deviceSynchronize()
    start = time.time()
    yield
    cupy.cuda.runtime.deviceSynchronize()
    end = time.time()
    print('%s:  %f sec' % (message, end - start))


def fit(X, n_clusters, max_iter):
    assert X.ndim == 2
    xp = cupy.get_array_module(X)
    pred = xp.zeros(len(X), dtype=np.int32)
    initial_indexes = np.random.choice(len(X), n_clusters,
                                       replace=False).astype(np.int32)
    centers = X[initial_indexes]

    for _ in six.moves.range(max_iter):
        # calculate distances and label
        distances = xp.linalg.norm(X[:, None, :] - centers[None, :, :],
                                   axis=2)

        new_pred = xp.argmin(distances, axis=1).astype(np.int32)
        if xp.all(new_pred == pred):
            break
        pred = new_pred

        # calculate centers
        centers = xp.stack([X[pred == i].mean(axis=0)
                            for i in six.moves.range(n_clusters)])
    return centers, pred


def run(gpuid, n_clusters, num, max_iter):
    samples = np.random.randn(num, 2).astype(np.float32)
    X_train = np.r_[samples + 1, samples - 1]
    repeat = 1

    with timer(' Numpy '):
        for i in range(repeat):
            centers, pred = fit(X_train, n_clusters, max_iter)

    with timer(' SciPy '):
        for i in range(repeat):
            centers, pred = kmeans(X_train, n_clusters, n_iter)

    with cupy.cuda.Device(gpuid):
        X_train = cupy.asarray(X_train)
        with timer(' CuPy '):
            for i in range(repeat):
                centers, pred = fit(X_train, n_clusters, max_iter)


run(gpu_id, n_clusters, n_samples, n_iter)

 Numpy :  12.553136 sec
 SciPy :  14.795404 sec
 CuPy :  1.217231 sec
